# Inroduction


This sample notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

Please note that the pricing model used in this notebook is a simple baseline. You are expected to design and implement a more advanced and effective model.


In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [ ]:
df = pd.read_csv('/content/dataset.csv')
places = df['SystemCodeNumber'].unique()
df

# You can find the sample dataset here: https://drive.google.com/file/d/1D479FLjp9aO3Mg8g6Lpj9oRViWacurA6/view?usp=sharing

In [ ]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True).drop(columns = 'ID')


In [ ]:
df

In [ ]:
# df['TrafficConditionNearby'].unique() # low/average/high
# df['VehicleType'].unique() # car,bike,truck,cycle
# df['IsSpecialDay'].unique() 0/1
# queuelenth 0-15

# Label encoding for TrafficConditionNearby
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].map({
    'low': 1,
    'average': 2,
    'high': 3
})

# Label encoding for VehicleType
df['VehicleType'] = df['VehicleType'].map({
    'cycle': 1,
    'bike': 2,
    'car': 3,
    'truck': 4
})

df

In [ ]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[["SystemCodeNumber","Timestamp", "Occupancy", "Capacity", "VehicleType", "TrafficConditionNearby", "QueueLength", "IsSpecialDay"]].to_csv("parking_stream.csv", index=False)

# Note: Only three features are used here for simplicity.
# Participants are expected to incorporate additional relevant features in their models.

In [ ]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    SystemCodeNumber : str
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity at the location
    VehicleType: int # Type of vehicle (1=cycle, 2=bike, 3=car, 4=truck)
    TrafficConditionNearby: int # Level of traffic (1=low, 2=average, 3=high)
    QueueLength: int # 0-15
    IsSpecialDay: int # 0/1


In [ ]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)


In [ ]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


# Step 2: Making a simple pricing function

In [ ]:
# Define a daily tumbling window over the data stream using Pathway
# This block performs temporal aggregation and computes a dynamic price for each day
import datetime

delta_window = (
    data_with_time
    .windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .groupby(pw.this.SystemCodeNumber)
    .reduce(
        SystemCodeNumber = pw.reducers.argmax(pw.this.t, pw.this.SystemCodeNumber),
        occupancy = pw.reducers.argmax(pw.this.t, pw.this.Occupancy),
        capacity = pw.reducers.argmax(pw.this.t, pw.this.Capacity),
        t = pw.reducers.argmax(pw.this.t, pw.this.t),
        day = pw.reducers.argmax(pw.this.t, pw.this.day),
        VehicleType = pw.reducers.argmax(pw.this.t, pw.this.VehicleType),
        TrafficConditionNearby = pw.reducers.argmax(pw.this.t, pw.this.TrafficConditionNearby),
        QueueLength = pw.reducers.argmax(pw.this.t, pw.this.QueueLength),
        IsSpecialDay = pw.reducers.argmax(pw.this.t, pw.this.IsSpecialDay),

    )
    .with_columns(
        # alpha = 2.0
        # beta = 1.5
        # gamma = 1.0
        # delta = 3.0
        # epsilon = 0.8
        # baseprice 10
        normalized_demand = (
                                2*(pw.this.occupancy/pw.this.capacity)
                             + 1.5*(pw.this.QueueLength)
                                  -(pw.this.TrafficConditionNearby)
                                +3*(pw.this.IsSpecialDay)
                             + 0.8*(pw.this.VehicleType)
                             )
                             /

                           (
                                  (pw.this.occupancy/pw.this.capacity)
                                +(pw.this.QueueLength)
                                +(pw.this.TrafficConditionNearby)
                                +(pw.this.IsSpecialDay)
                               + (pw.this.VehicleType)
                               )

    )
    .with_columns(

                   price = 10 *( 1 + pw.this.normalized_demand)
    )
)

delta_window


In [ ]:
pw.io.csv.write(delta_window, '/content/output.csv')

In [ ]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

In [ ]:
df_1 = pd.read_csv('/content/output.csv')
df_1['t'] = pd.to_datetime(df_1['t'])
df_1['price'] = df_1['price'].round(2)
df_1

In [ ]:
for place in places:
   mask = df_1['SystemCodeNumber'] == place
   df_2 = df_1[mask]
   data = df_2.copy().reset_index(drop=True)

#Update prices row-by-row using previous row
   for i in range(1, len(data)):
    prev_price = data.loc[i - 1, 'price']
    prev_demand = data.loc[i - 1, 'normalized_demand']
    demand = data.loc[i, 'normalized_demand']
    if (prev_demand) < (demand):
      alpha = 5
      demand_diff = demand - prev_demand
    else:
      alpha = -5
      demand_diff = prev_demand - demand

    data.loc[i, 'price'] = prev_price + alpha * (demand_diff)

   df_1.loc[mask, 'price'] = data['price'].values


df_1['price'] = df_1['price'].round(2)
df_1

# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [ ]:
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.io import curdoc
from bokeh.plotting import ColumnDataSource, figure, output_file, show


# print(df)

curdoc().theme = 'dark_minimal'


def plot_price_fluctuations(df, place):
   p = figure(title=place ,width=1000, height=400, x_axis_type="datetime")
   p.line(df.t, df.price)
   p.scatter(df.t, df.price, fill_color="red", size=5)
   show(p)


for place in places:
   plot_price_fluctuations(df_1[df_1['SystemCodeNumber'] == place], place)


In [ ]:
from bokeh.io import show, curdoc
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import Dropdown, CustomJS
from bokeh.layouts import column
from bokeh.models import Select

places = df_1['SystemCodeNumber'].unique().tolist()
df_1['t_ms'] = pd.to_datetime(df_1['t']).astype('int64') // 10**6  # to milliseconds

data_dict = {
    place: df_1[df_1['SystemCodeNumber'] == place][['t_ms', 'price']].to_dict('list')
    for place in places
}

#Initialize with the first place
initial_place = places[0]
source = ColumnDataSource(data={
    't_ms': data_dict[initial_place]['t_ms'],
    'price': data_dict[initial_place]['price'],
})

#Create plot
curdoc().theme = 'dark_minimal'

p = figure(title=f"{initial_place}", x_axis_type="datetime", width=1000, height=400)
p.line('t_ms', 'price', source=source)
p.scatter('t_ms', 'price', source=source, fill_color='red', size=5)

#Select menu
menu = [(place, place) for place in places]
select = Select(title="Select Place", value=initial_place, options=places)
select.js_on_change("value", CustomJS(args=dict(source=source, full_data=data_dict, plot=p), code="""
    const selected = this.value;
    const data = full_data[selected];
    source.data = { t_ms: data.t_ms, price: data.price };
    plot.title.text = selected;
    source.change.emit();
"""))


layout = column(select, p)
show(layout)

#Output Html File
from bokeh.io import output_file

output_file("price_plot.html", title="Dynamic Price Plot by Location")